## Pipistrel Data Downloader

### Selenium Data_Downloader

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from zipfile import ZipFile
from zipfile import BadZipFile
from datetime import datetime
import time
import os 
import requests
import io

In [2]:
login_url = 'https://cloud.pipistrel.si/electro/login'
# username = 'paulparker'
# password = 'dfh5fg616'
username = 'nancy_marshall'
password = 'yr4EJvXu8YMc6ti'

In [7]:
driver = webdriver.Chrome()
driver.get(login_url)

In [8]:
username_field = driver.find_element("name", "username")
password_field = driver.find_element("name", "password")
username_field.send_keys(username)
password_field.send_keys(password)
password_field.send_keys(Keys.RETURN)
time.sleep(3)

In [6]:
output_folder = os.path.join(r'C:\Users\Nathan\Documents\Backups\Vancouver') #<-- change it to your folder's name 

In [9]:
base_url = 'https://cloud.pipistrel.si/electro/flight'
start_id = 6800  # Update flight id number to where you want to start download 
final_id = 7003  # Update this with the final flight ID you want to reach
current_id = start_id
while current_id <= final_id:
    page_url = f"{base_url}/{current_id}"
    driver.get(page_url)
    csv_links = driver.find_elements(By.LINK_TEXT, "Download CSV file")
    for csv_link in csv_links:
        file_name = os.path.basename(csv_link.get_attribute('href'))
        table = driver.find_element(By.CLASS_NAME, "white-link")
        rows = table.find_elements(By.TAG_NAME, "tr")
        flight_info = {}

        for row in rows:
            column = row.find_elements(By.TAG_NAME, "td")
            if len(column) == 2:
                key = column[0].text.strip().replace(" ", "_")
                value = column[1].text.strip()
                flight_info[key] = value

        try:
            date_time_obj = datetime.strptime(flight_info['Date'].replace('.', ''), "%b %d, %Y, %I:%M %p" if ':' in flight_info['Date'] else "%b %d, %Y, %I %p")
        except ValueError:
            try:
                date_time_obj = datetime.strptime(flight_info['Date'].replace('.', ''), "%B %d, %Y, %I:%M %p" if ':' in flight_info['Date'] else "%B %d, %Y, %I %p")
            except ValueError:
                pass
            
        formatted_date = date_time_obj.strftime("%b%d").lower()
        formatted_time = date_time_obj.strftime("%I%M%p").lower()

        flight_type = flight_info.get('Flight_type', '').lower()
        type_abbreviation = 'ch' if flight_type == 'charging' else 'gt' if flight_type == 'ground test' else 'dc'
        new_file_name = f"{file_name.split('.')[0]}_{formatted_date}_{formatted_time}_{type_abbreviation}.zip"
        new_file_path = os.path.join(output_folder, new_file_name)

        # Download the zip file
        response = requests.get(csv_link.get_attribute('href'))
        try:
            with ZipFile(io.BytesIO(response.content), 'r') as zip_ref:
                # Provide the directory where you want to extract the contents
                extraction_path = os.path.join(output_folder, f"{file_name.split('.')[0]}_{formatted_date}_{formatted_time}_{type_abbreviation}")
                zip_ref.extractall(extraction_path)
                print(f"Unzipped contents to: {extraction_path}")
        except BadZipFile:
            pass

    current_id += 1
driver.quit()

Unzipped contents to: C:\Users\Nathan\Documents\Backups\Vancouver\csv-27717-6838_jul08_0900pm_ch
Unzipped contents to: C:\Users\Nathan\Documents\Backups\Vancouver\csv-27719-6839_jul08_0948pm_dc
Unzipped contents to: C:\Users\Nathan\Documents\Backups\Vancouver\csv-27721-6840_jul09_0400pm_ch
Unzipped contents to: C:\Users\Nathan\Documents\Backups\Vancouver\csv-27723-6841_jul09_0508pm_dc
Unzipped contents to: C:\Users\Nathan\Documents\Backups\Vancouver\csv-27725-6842_jul09_0600pm_ch
Unzipped contents to: C:\Users\Nathan\Documents\Backups\Vancouver\csv-27727-6843_jul09_0724pm_dc
Unzipped contents to: C:\Users\Nathan\Documents\Backups\Vancouver\csv-27729-6844_jul10_0600pm_ch
Unzipped contents to: C:\Users\Nathan\Documents\Backups\Vancouver\csv-27731-6845_jul10_0713pm_dc
Unzipped contents to: C:\Users\Nathan\Documents\Backups\Vancouver\csv-27733-6846_jul10_0742pm_dc
Unzipped contents to: C:\Users\Nathan\Documents\Backups\Vancouver\csv-27735-6847_jul10_0742pm_ch
Unzipped contents to: C:\Users